In [1]:
import pandas as pd
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from sklearn.preprocessing import normalize

In [2]:
ftdict = {'from': [], 'to': []}
idval = {'ID': [], 'myvalue': []}

placeID_list = []
cuisine_list = []
userID_list = []
payment_list = []
parking_lot_list = []
ThetaDict = []
# Restaurant - Cuisine
with open("RCdata/chefmozcuisine.csv", 'r', encoding="UTF-8") as page:
    page.readline()
    for line in page:
        placeID, Rcuisine = line.strip().split(',')
        placeID = placeID
        ftdict['from'].append(placeID)
        ftdict['to'].append(Rcuisine)
        ftdict['from'].append(Rcuisine)
        ftdict['to'].append(placeID)
        if placeID not in placeID_list:
            placeID_list.append(placeID)
        if Rcuisine not in cuisine_list:
            cuisine_list.append(Rcuisine)

ThetaDict.append((2, 3))
ThetaDict.append((3, 2))

# User - Payment
with open("RCdata/userpayment.csv", 'r', encoding="UTF-8") as page:
    page.readline()
    for line in page:
        userID, Upayment = line.strip().split(',')
        ftdict['from'].append(userID)
        ftdict['to'].append(Upayment)
        ftdict['from'].append(Upayment)
        ftdict['to'].append(userID)
        if userID not in userID_list:
            userID_list.append(userID)
        if Upayment not in payment_list:
            payment_list.append(Upayment)
        
ThetaDict.append((1, 4))
ThetaDict.append((4, 1))

#User - Cuisine
with open("RCdata/usercuisine.csv", 'r', encoding="UTF-8") as page:
    page.readline()
    for line in page:
        userID, Rcuisine = line.strip().split(',')
        ftdict['from'].append(userID)
        ftdict['to'].append(Rcuisine)
        ftdict['from'].append(Rcuisine)
        ftdict['to'].append(userID)
        if userID not in userID_list:
            userID_list.append(userID)
        if Rcuisine not in cuisine_list:
            cuisine_list.append(Rcuisine)

ThetaDict.append((1, 3))
ThetaDict.append((3, 1))

# Payment - Restaurant
with open("RCdata/chefmozaccepts.csv", 'r', encoding="UTF-8") as page:
    page.readline()
    for line in page:
        placeID, Rpayment = line.strip().split(',')
        ftdict['from'].append(placeID)
        ftdict['to'].append(Rpayment)
        ftdict['from'].append(Rpayment)
        ftdict['to'].append(placeID)
        if placeID not in placeID_list:
            placeID_list.append(placeID)
        if Rpayment not in payment_list:
            payment_list.append(Rpayment)
            
ThetaDict.append((2, 4))
ThetaDict.append((4, 2))

# Restaurant - Parking Lot
with open("RCdata/chefmozparking.csv", 'r', encoding="UTF-8") as page:
    page.readline()
    for line in page:
        placeID, parking_lot = line.strip().split(',')
        ftdict['from'].append(placeID)
        ftdict['to'].append(parking_lot)
        ftdict['from'].append(parking_lot)
        ftdict['to'].append(placeID)
        if placeID not in placeID_list:
            placeID_list.append(placeID)
        if parking_lot not in parking_lot_list:
            parking_lot_list.append(parking_lot)
            
ThetaDict.append((2, 5))
ThetaDict.append((5, 2))

ur_edges = []
# User - Restaurant
with open("RCdata/rating_final.csv", 'r', encoding="UTF-8") as page:
    page.readline()
    for line in page:
        userID, placeID, rating, food_rating, service_rating = line.strip().split(',')
        if rating == '2':
            ur_edges.append((userID, placeID))
            ftdict['from'].append(userID)
            ftdict['to'].append(placeID)
            ftdict['from'].append(placeID)
            ftdict['to'].append(userID)
            if userID not in userID_list:
                userID_list.append(userID)
            if placeID not in placeID_list:
                placeID_list.append(placeID)
                
ThetaDict.append((1, 2))
ThetaDict.append((2, 1))

for userID in userID_list:
    idval['ID'].append(userID)
    idval['myvalue'].append(1)
    
for placeID in placeID_list:
    idval['ID'].append(placeID)
    idval['myvalue'].append(2)

for cuisine in cuisine_list:
    idval['ID'].append(cuisine)
    idval['myvalue'].append(3)

for payment in payment_list:
    idval['ID'].append(payment)
    idval['myvalue'].append(4)

for parking_lot in parking_lot_list:
    idval['ID'].append(parking_lot)
    idval['myvalue'].append(5)
        
node_type_set = set(idval['myvalue'])
print(node_type_set)

# Build a dataframe with your connections
df = pd.DataFrame(ftdict)

# And a data frame with characteristics for your nodes
carac = pd.DataFrame(idval)

# Build your graph
G = nx.from_pandas_edgelist(df, 'from', 'to', create_using=nx.Graph())

# The order of the node for networkX is the following order:
Gnodes = list(G.nodes())
Gnode_types = [idval['myvalue'][idval['ID'].index(node)] for node in Gnodes]
#for node in Gnodes:
    #print(node, idval['myvalue'][idval['ID'].index(node)])

#print(G.nodes())
#print(G.edges(), len(G.edges()))
A = nx.adjacency_matrix(G)
#print(A)
# Thus, we cannot give directly the 'myvalue' column to netowrkX, we need to arrange the order!

# Here is the tricky part: I need to reorder carac to assign the good color to each node
carac = carac.set_index('ID')
carac = carac.reindex(G.nodes())

# And I need to transform my categorical column in a numerical value: group1->1, group2->2...
carac['myvalue'] = pd.Categorical(carac['myvalue'])
#carac['myvalue'].cat.codes

{1, 2, 3, 4, 5}


In [3]:
assert(len(idval['ID']) == len(idval['myvalue']))
with open("RC-metadata.txt", "w") as wf:
    for i in range(len(idval['ID'])):
        wf.write("%d\t%s\t%s\n"%(Gnodes.index(idval['ID'][i]), idval['ID'][i], idval['myvalue'][i]))

nA = A.toarray()
with open("RC-edges.txt", "w") as wf:
    for (n1, n2) in G.edges():
        wf.write("%d\t%d\n"%(Gnodes.index(n1), Gnodes.index(n2)))

In [3]:
nA = A.toarray()

AUC = 0.0
U = 0
for u in userID_list:
    u_neighbors = G.neighbors(u)
    j_list = set(placeID_list) - set(u_neighbors)
    i_list = set(placeID_list) - j_list
    if len(i_list) < 1:
        continue

    U += 1
    for i in i_list:
        tA = nA
        tA[Gnodes.index(u)][Gnodes.index(i)] = tA[Gnodes.index(i)][Gnodes.index(u)] = 0
        P = normalize(tA, norm='l1', axis=0)

        r0 = np.zeros((len(Gnodes), 1))
        r0[Gnodes.index(u)] = 1
        
        r = r0
        c = 0.2
        #for _ in range(1000):
            #r = (1-c)*np.dot(P, r) + c*r0
        r = c*np.dot(np.linalg.inv(np.identity(len(Gnodes)) - (1-c)*P),r0)
        
        j_scores = [r[Gnodes.index(j)][0] for j in j_list]
        i_score = r[Gnodes.index(i)][0]
        
        for j_score in j_scores:
            if i_score > j_score:
                AUC += 1/(len(j_list)*len(i_list))

AUC /= U
print(AUC) # 0.8116120020513362

0.8154576279009331


In [96]:
import random
import math

def node_idx(nodeID):
    return Gnodes.index(nodeID)
    
def node_type(nidx):
    return Gnode_types[nidx]

def D_count(n_from, n_to):
    D = R[:, n_from].tolist()
    D_count = 0
    for z in range(len(D)):
        if D[z] == 1 and node_type(z) == node_type(n_to):
            assert(z != n_from)
            D_count += 1
    return D_count

def edge_type(n_from, n_to):
    edge = (node_type(n_from), node_type(n_to))
    if edge not in ThetaDict:
        return -1
    return ThetaDict.index(edge)

iur_edges = []
for (u, i) in ur_edges:
    iur_edges.append((Gnodes.index(u), Gnodes.index(i)))

train_ratio = 0.5
train_edges = random.sample(iur_edges, int(len(iur_edges)*train_ratio))
test_edges = list(set(iur_edges) - set(train_edges))

c = 0.2
alpha = 1-c
#user_idx = [Gnodes.index(u) for u in userID_list]
restaurant_idx = [Gnodes.index(i) for i in placeID_list]


print("Construct Type Matrix...")
R = nx.adjacency_matrix(G).toarray()
for edge in test_edges:
    n1, n2 = edge
    assert(R[n1, n2] == 1)
    assert(R[n2, n1] == 1)
    R[n1, n2] = -1
    R[n2, n1] = -1
    
T = np.zeros(R.shape)
for i in range(R.shape[0]):
    for j in range(R.shape[1]):
        if R[i, j] == 1:
            T[i, j] = edge_type(j, i)
        else:
            T[i, j] = -1

def reset_P(P, Theta):
    for i in range(R.shape[0]):
        for j in range(R.shape[1]):
            if R[i, j] != 1:
                P[i, j] = 0
                continue
            tidx = int(T[i, j]) #edge_type(i, j)
            assert(tidx >= 0)
            d = D_count(j, i)
            assert(d != 0)
            P[i, j] = Theta[tidx] / d
    P = normalize(P, norm='l1', axis=0)

def getP(i, j, Theta):
    if R[j, i] != 1:
        return 0
    tidx = int(T[j, i]) #edge_type(i, j)
    assert(tidx >= 0)
    d = D_count(i, j)
    assert(d != 0)
    return Theta[tidx] / d

def normalizeTheta(Theta):
    for p in node_type_set:
        sum_p = 0
        idx_p = []
        for q in node_type_set:
            if (p, q) in ThetaDict:
                idx = ThetaDict.index((p, q))
                sum_p += Theta[idx]
                idx_p.append(idx)
        if sum_p != 0:
            for ix in idx_p:
                Theta[ix] /= sum_p

def calculateBPR(r):
    BPR = 0
    for (xu, xi) in train_edges:
        for xj in range(R.shape[0]):
            if R[xj, xu] == 0:
                BPR += math.log(1/(1 + math.exp(-(r[xi, xu] - r[xj, xu]))))
    return BPR
                
Theta = [1]*len(ThetaDict)
bestTheta = None
bestScore = -9999999999999
def HeteLearn(Adj, P):
    global bestScore
    global bestTheta
    # Algorithm: HeteLearn
    # Randomly initialize Theta
    #for i in range(len(ThetaDict)):
    #    Theta.append(random.random())
    global Theta
    normalizeTheta(Theta)

    # repeat until coverage
    iter_count = 0
    converge_count = 0
    reset_P(P, Theta)
    
    #r0 = np.identity(len(Gnodes))
    r = c*np.linalg.inv(np.identity(len(Gnodes)) - (1-c)*P)
    print(calculateBPR(r))
    while True:
        u, i = random.sample(train_edges, 1)[0]
        iter_count += 1
        
        gradTheta = []
        path_i = 0
        path_j = 0
        for ta in range(len(ThetaDict)):
            ################################################################
            drui = 0
            for q in range(len(Gnodes)):
                if R[q, u] != 1: continue # from q to u
                Puq = P[q, u] #getP(u, q, Theta)

                for p in range(len(Gnodes)):
                    if R[p, q] != 1: continue
                    Pqp = P[p, q] #getP(q, p, Theta)

                    if R[i, p] == 1:
                        Ppi = P[i, p] #getP(p, i, Theta)
                        drui += alpha**3*(
                            Puq*Pqp*1/D_count(p, i) if T[i, p] == ta else 0 +
                            Puq*1*Ppi/D_count(q, p) if T[p, q] == ta else 0 +
                            1*Pqp*Ppi/D_count(u, q) if T[q, u] == ta else 0
                        )
                        path_i += 3

                if R[i, q] == 1:
                    Pqi = P[i, q] #getP(q, i, Theta)
                    drui += alpha**2 * (1-alpha) * (
                        Puq*1/D_count(q, i) if T[i, q] == ta else 0 + 
                        1*Pqi/D_count(u, q) if T[q, u] == ta else 0
                    )
                    path_i += 2

            if R[i, u] == 1 and T[i, u] == ta: # from u to i
                drui += alpha * (1-alpha) / D_count(u, i)
                path_i += 1

            #################################################################
            
            gradTa = 0
            j = -1
            while j < 0:
                j = random.sample(restaurant_idx, 1)[0]
                if R[j, u] == 1: continue
                break
                    
            druj = 0
            for q in range(len(Gnodes)):
                if R[q, u] != 1: continue # from q to u
                Puq = P[q, u] #getP(u, q, Theta)

                for p in range(len(Gnodes)):
                    if R[p, q] != 1: continue
                    Pqp = P[p, q] #getP(q, p, Theta)

                    if R[j, p] == 1:
                        Ppj = P[j, p] #getP(p, j, Theta)
                        druj += alpha**3*(
                            Puq*Pqp*1/D_count(p, j) if T[j, p] == ta else 0 +
                            Puq*1*Ppj/D_count(q, p) if T[p, q] == ta else 0 +
                            1*Pqp*Ppj/D_count(u, q) if T[q, u] == ta else 0
                        )
                        path_j += 3

                if R[j, q] == 1:
                    Pqj = P[j, q] #getP(q, j, Theta)
                    druj += alpha**2 * (1-alpha) * (
                        Puq*1/D_count(q, j) if T[j, q] == ta else 0 + 
                        1*Pqj/D_count(u, q) if T[q, u] == ta else 0
                    )
                    path_j += 2

            rui = r[i][u]
            ruj = r[j][u]
            gradTa += (drui - druj) / (1 + math.exp((rui - ruj)))
                
            gradTheta.append(gradTa)
            
        ##########################################
        gradTheta = np.array(gradTheta)
        learning_rate = 10.0
        while min(Theta + learning_rate*gradTheta) < 0:
            learning_rate /= 2

        NewTheta = Theta + learning_rate*gradTheta
        normalizeTheta(NewTheta)
        
        P2 = P
        reset_P(P2, NewTheta)
        r2 = c*np.linalg.inv(np.identity(len(Gnodes)) - (1-c)*P2)
              
        BPR_score = calculateBPR(r2)
        print("Iteration:", iter_count, "BPR:", BPR_score, "~ visited paths:", path_i + path_j)
        if bestScore < BPR_score:
            bestScore = BPR_score
            bestTheta = np.copy(NewTheta)
        
        if iter_count >= 150:#10000: # and converge_count >= 10:
            Theta = NewTheta
            P = P2
            r = r2
            break
        Theta = NewTheta
        P = P2
        r = r2

P = np.zeros(R.shape)
print("Hetelearn...")
HeteLearn(R, P)

Construct Type Matrix...
Hetelearn...
-196423.27692688737
Iteration: 1 BPR: -194892.7769210736 ~ visited paths: 883
Iteration: 2 BPR: -194016.9505369387 ~ visited paths: 1136
Iteration: 3 BPR: -193506.9104936553 ~ visited paths: 2352
Iteration: 4 BPR: -191940.67144358082 ~ visited paths: 610
Iteration: 5 BPR: -191594.34704430183 ~ visited paths: 1198
Iteration: 6 BPR: -190573.13089385352 ~ visited paths: 1215
Iteration: 7 BPR: -188981.31619975547 ~ visited paths: 727
Iteration: 8 BPR: -187852.496197955 ~ visited paths: 729
Iteration: 9 BPR: -186173.03334635106 ~ visited paths: 474
Iteration: 10 BPR: -185804.60623095368 ~ visited paths: 981
Iteration: 11 BPR: -186764.67074545234 ~ visited paths: 970
Iteration: 12 BPR: -186003.26880990423 ~ visited paths: 582
Iteration: 13 BPR: -186019.3730570158 ~ visited paths: 722
Iteration: 14 BPR: -185897.93379181862 ~ visited paths: 1539
Iteration: 15 BPR: -184501.486419882 ~ visited paths: 397
Iteration: 16 BPR: -185131.57905694007 ~ visited paths

Iteration: 136 BPR: -185186.72966636173 ~ visited paths: 732
Iteration: 137 BPR: -185138.8178089498 ~ visited paths: 981
Iteration: 138 BPR: -185594.79497357176 ~ visited paths: 1311
Iteration: 139 BPR: -185603.53689734926 ~ visited paths: 1123
Iteration: 140 BPR: -186485.80955656886 ~ visited paths: 1064
Iteration: 141 BPR: -185309.39673857114 ~ visited paths: 634
Iteration: 142 BPR: -185011.3675878045 ~ visited paths: 834
Iteration: 143 BPR: -187242.03946686804 ~ visited paths: 465
Iteration: 144 BPR: -186685.3700701723 ~ visited paths: 1166
Iteration: 145 BPR: -184770.28880299305 ~ visited paths: 344
Iteration: 146 BPR: -185562.89315960286 ~ visited paths: 1112
Iteration: 147 BPR: -185578.17025002273 ~ visited paths: 945
Iteration: 148 BPR: -185090.03915877474 ~ visited paths: 451
Iteration: 149 BPR: -186444.83236599647 ~ visited paths: 2032
Iteration: 150 BPR: -187199.8810729101 ~ visited paths: 1895


In [90]:
ThetaG = [1]*len(ThetaDict)
bestThetaG = None
bestScoreG = -9999999999999

def HeteLearnGeneral(Adj, P):
    global bestScoreG
    global bestThetaG
    # Algorithm: HeteLearn
    # Randomly initialize Theta
    #for i in range(len(ThetaDict)):
    #    Theta.append(random.random())
    global ThetaG
    normalizeTheta(ThetaG)

    # repeat until coverage
    iter_count = 0
    converge_count = 0
    reset_P(P, ThetaG)
    
    #r0 = np.identity(len(Gnodes))
    r = c*np.linalg.inv(np.identity(len(Gnodes)) - (1-c)*P)
    print(calculateBPR(r))
    while True:
        u, i = random.sample(train_edges, 1)[0]
        iter_count += 1
        
        gradTheta = []
        r = c*np.linalg.inv(np.identity(len(Gnodes)) - (1-c)*P)
        for ta in range(len(ThetaDict)):
            _P = np.copy(P)
            for ii in range(P.shape[0]):
                for jj in range(P.shape[1]):
                    if T[ii, jj] == ta:
                        _P[ii, jj] = 0

            _r = c*np.linalg.inv(np.identity(len(Gnodes)) - (1-c)*_P)
            Dr = (r - _r)/ThetaG[ta]
            
            drui = Dr[i][u]
            druj = Dr[j][u]
            rui = r[i][u]
            ruj = r[j][u]
            gradTa = (drui - druj) / (1 + math.exp((rui - ruj)))
                
            gradTheta.append(gradTa)
            
        ##########################################
        gradTheta = np.array(gradTheta)
        learning_rate = 10.0
        while min(ThetaG + learning_rate*gradTheta) < 0:
            learning_rate /= 2

        NewThetaG = ThetaG + learning_rate*gradTheta
        normalizeTheta(NewThetaG)
        
        P2 = P
        reset_P(P2, NewThetaG)
        r2 = c*np.linalg.inv(np.identity(len(Gnodes)) - (1-c)*P2)
                
        BPR_score = calculateBPR(r2)
        print(BPR_score)
        if bestScoreG < BPR_score:
            bestScoreG = BPR_score
            bestThetaG = NewThetaG
        
        if iter_count >= 150:#10000: # and converge_count >= 10:
            ThetaG = NewThetaG
            P = P2
            r = r2
            break
        ThetaG = NewThetaG
        P = P2
        r = r2

P = np.zeros(R.shape)
print("Hetelearn...")
HeteLearnGeneral(R, P)

Hetelearn...
-196496.29483730442
-194624.7564003863
-193856.1158458084
-192735.49406823708
-191766.8905830253
-191270.0270301035
-190919.70106743724
-190243.88016485152
-189873.95253308146
-190002.32489652303
-189689.83090975918
-189644.5768464254
-189234.7409003788
-188863.6708014249
-189293.00807030668
-189034.06020619502
-188724.85715216983
-188417.75935807245
-188667.0587458039
-188241.97956585628
-186783.12232837194
-186915.27698113566
-187274.9350575519
-187335.84323895004
-187552.26572634318
-187882.07853506797
-187633.50763278466
-187295.17345718824
-187302.44973654865
-187121.21829712394
-186032.38238098705
-186558.51676901555
-186696.64740419693
-186578.37236151294
-186910.71673550172
-188540.7106527146
-188172.9303572131
-187954.36192045265
-187967.24216236148
-187910.55700733542
-187437.81842842727
-187611.67161382904
-187863.68439783438
-187426.53056740892
-187470.33490213333
-187378.6630901915
-187397.29154141864
-187194.436210969
-187221.6630744074
-187376.7125004413
-18

In [91]:
for edge in test_edges:
    n1, n2 = edge
    R[n1, n2] = 0 #1
    R[n2, n1] = 0 #1 for recover

for i in range(R.shape[0]):
    for j in range(R.shape[1]):
        if R[i, j] == 1:
            T[i, j] = edge_type(j, i)
        else:
            T[i, j] = -1
print("Start Evaluation")

reset_P(P, bestTheta)
P3 = np.copy(P)
reset_P(P3, bestThetaG)
P2 = normalize(R, norm='l1', axis=0)

AUC = 0.0
AUC2 = 0.0
AUC3 = 0.0
E = 0

tmpDict = {}

for _, (u, i) in enumerate(test_edges):
    u_neighbors = None
    if u in tmpDict:
        u_neighbors = tmpDict[u]
    else:
        u_neighbors = [Gnodes.index(un) for un in G.neighbors(Gnodes[u])]
        tmpDict[u] = u_neighbors
    
    j_list = set(restaurant_idx) - set(u_neighbors)
    
    #assert(R[Gnodes.index(u), Gnodes.index(i)] == 1)
    #assert(R[Gnodes.index(i), Gnodes.index(u)] == 1)
    #R[Gnodes.index(u), Gnodes.index(i)] = R[Gnodes.index(i), Gnodes.index(u)] = 0
    #tmp1 = T[Gnodes.index(u), Gnodes.index(i)]
    #tmp2 = T[Gnodes.index(i), Gnodes.index(u)]
    #T[Gnodes.index(u), Gnodes.index(i)] = T[Gnodes.index(i), Gnodes.index(u)] = -1
    
    # Prepare P and _P
    #reset_P(P, Theta)
    r0 = np.zeros((len(Gnodes), 1))
    r0[u] = 1
    #P2 = normalize(R, norm='l1', axis=0)

    j_correct = 0
    j2_correct = 0
    j3_correct = 0

    c = 0.2
    r  = c*np.dot(np.linalg.inv(np.identity(len(Gnodes)) - (1-c)*P),r0)
    r2 = c*np.dot(np.linalg.inv(np.identity(len(Gnodes)) - (1-c)*P2),r0)
    r3 = c*np.dot(np.linalg.inv(np.identity(len(Gnodes)) - (1-c)*P3),r0)
    
    j_scores = [r[j][0] for j in j_list]
    i_score = r[i][0]
    for j_score in j_scores:
        if i_score > j_score:
            j_correct += 1

    # for r2
    j2_scores = [r2[j][0] for j in j_list]
    i2_score = r2[i][0]
    for j2_score in j2_scores:
        if i2_score > j2_score:
            j2_correct += 1
    
    
    # for r3
    j3_scores = [r3[j][0] for j in j_list]
    i3_score = r3[i][0]
    for j3_score in j3_scores:
        if i3_score > j3_score:
            j3_correct += 1

    #print(i_score, " vs. ", i2_score, " ~ ", i_score - i2_score)

    AUC += j_correct / len(j_list)
    AUC2 += j2_correct / len(j_list)
    AUC3 += j3_correct / len(j_list)

    #R[Gnodes.index(u), Gnodes.index(i)] = R[Gnodes.index(i), Gnodes.index(u)] = 1
    #T[Gnodes.index(u), Gnodes.index(i)] = tmp1
    #T[Gnodes.index(i), Gnodes.index(u)] = tmp2

    E += 1
    print(E, "...", _+1, "/", len(test_edges), "~", AUC/E, AUC2/E, AUC3/E)
    #print("AUC %d: "%U, AUC/U)

assert(E == len(test_edges))
AUC /= E
AUC2 /= E
AUC3 /= E
print("AUC:", AUC)
print("AUC2:", AUC2)
print("AUC3:", AUC3) 
#AUC: 0.9512128165675547
#AUC2: 0.9081488700135977

Start Evaluation
1 ... 1 / 243 ~ 0.9870689655172413 0.9730603448275862 0.9870689655172413
2 ... 2 / 243 ~ 0.9752941394109812 0.9564872539588575 0.9785130235311528
3 ... 3 / 243 ~ 0.9110500577773961 0.8992297516525142 0.9110431279945755
4 ... 4 / 243 ~ 0.910682591772229 0.8894384601333577 0.9122920338323148
5 ... 5 / 243 ~ 0.9255353207296112 0.9001529186443206 0.9268228743776799
6 ... 6 / 243 ~ 0.9264888861432761 0.9039047325974422 0.9273828285406552
7 ... 7 / 243 ~ 0.9360678227529787 0.9114816207132312 0.9368340590935895
8 ... 8 / 243 ~ 0.9440593449088563 0.9221396936121468 0.9447298017068908
9 ... 9 / 243 ~ 0.9266548238187183 0.8940349939844775 0.9320712240768891
10 ... 10 / 243 ~ 0.9052339766299795 0.8678289194787336 0.9065679643301443
11 ... 11 / 243 ~ 0.8784413443417293 0.8290251199318959 0.8722408727620898
12 ... 12 / 243 ~ 0.880621964717682 0.8087964278189573 0.8749381991030125
13 ... 13 / 243 ~ 0.886568644553937 0.795622891300148 0.8809901690168377
14 ... 14 / 243 ~ 0.8496916438

111 ... 111 / 243 ~ 0.90859181271418 0.8612058871806783 0.9074403582706162
112 ... 112 / 243 ~ 0.9092828802730744 0.8623873924983385 0.9081609492835941
113 ... 113 / 243 ~ 0.9097589928793074 0.8623272540523346 0.9086565991227437
114 ... 114 / 243 ~ 0.9104936816568006 0.8634874938816526 0.9094009580209084
115 ... 115 / 243 ~ 0.9105894355236465 0.8623089677543975 0.9094781633415308
116 ... 116 / 243 ~ 0.9112213222187786 0.8634033633896123 0.9101659279803047
117 ... 117 / 243 ~ 0.9117325078657844 0.8633786767064465 0.9106861340908872
118 ... 118 / 243 ~ 0.912471395522417 0.8645182006555179 0.9114247473841582
119 ... 119 / 243 ~ 0.9131798223965848 0.865566344446883 0.9121419696208324
120 ... 120 / 243 ~ 0.9106147934106635 0.8655217503578542 0.9099977757879707
121 ... 121 / 243 ~ 0.9110418156366796 0.866125517596276 0.9104031803197885
122 ... 122 / 243 ~ 0.9072477363844794 0.866010159549201 0.9070480715762405
123 ... 123 / 243 ~ 0.9079137366074808 0.8670202329466244 0.9077156950903656
124 .

218 ... 218 / 243 ~ 0.9087105606235317 0.8633076742572008 0.9093978502681631
219 ... 219 / 243 ~ 0.9090238540191085 0.8636507667236338 0.9097030742425001
220 ... 220 / 243 ~ 0.9071548584444136 0.8637415391005954 0.9082375351903007
221 ... 221 / 243 ~ 0.9073701820682413 0.8626027491748789 0.9084235800458915
222 ... 222 / 243 ~ 0.9046149061342905 0.8611873633532542 0.9060752611006107
223 ... 223 / 243 ~ 0.9047822629851276 0.8606526015618539 0.9061155234429435
224 ... 224 / 243 ~ 0.9050193214955978 0.8608600784736564 0.9063466298977986
225 ... 225 / 243 ~ 0.9030758664466568 0.8601881555442394 0.904521528985947
226 ... 226 / 243 ~ 0.9034760019646207 0.8607828484161179 0.9049152677670113
227 ... 227 / 243 ~ 0.9037537406263878 0.8602163223239144 0.9051819087221293
228 ... 228 / 243 ~ 0.9041522928864212 0.8605039983295679 0.9055883453609165
229 ... 229 / 243 ~ 0.9045614305925338 0.8610755062422983 0.9059912120955248
230 ... 230 / 243 ~ 0.9048870741963031 0.8615150134419499 0.9063012385529238


In [95]:
ThetaG = [1]*len(ThetaDict)
bestThetaGG = None
bestScoreGG = -9999999999999

def HeteLearnGeneralG(Adj, P):
    global bestScoreGG
    global bestThetaGG
    # Algorithm: HeteLearn
    # Randomly initialize Theta
    #for i in range(len(ThetaDict)):
    #    Theta.append(random.random())
    global ThetaG
    normalizeTheta(ThetaG)

    # repeat until coverage
    iter_count = 0
    converge_count = 0
    reset_P(P, ThetaG)
    
    #r0 = np.identity(len(Gnodes))
    r = c*np.linalg.inv(np.identity(len(Gnodes)) - (1-c)*P)
    h = 0.01
    print(calculateBPR(r))
    
    while True:
        u, i = random.sample(train_edges, 1)[0]
        iter_count += 1
        
        gradTheta = []
        r = c*np.linalg.inv(np.identity(len(Gnodes)) - (1-c)*P)
        for ta in range(len(ThetaDict)):
            Theta[ta] += h
            reset_P(P, Theta)
            r_h = c*np.linalg.inv(np.identity(len(Gnodes)) - (1-c)*P)
            Dr = (r_h - r)/h
            
            drui = Dr[i][u]
            druj = Dr[j][u]
            rui = r[i][u]
            ruj = r[j][u]
            gradTa = (drui - druj) / (1 + math.exp((rui - ruj)))
                
            gradTheta.append(gradTa)
            Theta[ta] -= h
            
        ##########################################
        gradTheta = np.array(gradTheta)
        learning_rate = 1000.0
        while min(ThetaG + learning_rate*gradTheta) < 0:
            learning_rate /= 2

        NewThetaG = ThetaG + learning_rate*gradTheta
        normalizeTheta(NewThetaG)
        
        P2 = P
        reset_P(P2, NewThetaG)
        r2 = c*np.linalg.inv(np.identity(len(Gnodes)) - (1-c)*P2)
                
        BPR_score = calculateBPR(r2)
        print(BPR_score)
        if bestScoreGG < BPR_score:
            bestScoreGG = BPR_score
            bestThetaGG = NewThetaG
        
        if iter_count >= 150:#10000: # and converge_count >= 10:
            ThetaG = NewThetaG
            P = P2
            r = r2
            break
        ThetaG = NewThetaG
        P = P2
        r = r2

P = np.zeros(R.shape)
print("Hetelearn...")
HeteLearnGeneralG(R, P)

Hetelearn...
-196950.5812493422
-196893.7197802548
-196886.17302217765
-196898.10205958612
-196889.79424879522
-196899.14866429128
-196885.66736546773
-196898.03936919468
-196899.3141539408
-196900.06358964168
-196904.5506500278
-196896.94632009568
-196904.96256141263
-196899.69840443137
-196899.76368833036
-196901.9601862653
-196898.94779547298
-196884.66261599434
-196898.84346706636
-196905.71313136435
-196900.53639949093
-196895.05642743263
-196902.5183191441
-196895.41269344106
-196895.84729118587
-196890.33468388702
-196898.12709248494
-196899.61022019104
-196900.12869894237
-196901.59626669346
-196897.4088386105
-196888.16027291704
-196904.34919655597
-196900.52833288067
-196897.64639090924
-196893.62344613936
-196892.71428106588
-196883.62561297897
-196902.94928446758
-196885.0942983265
-196899.80598950016
-196902.9322398932
-196899.96289896744
-196903.93119149923
-196897.9704657727
-196902.74947998425
-196901.22603798076
-196902.7700640941
-196897.3547828407
-196900.9831390564
